In [1]:
import pandas as pd

In [10]:
# traffic flow dataframe
tf_df = pd.read_csv("../data/interim/consolidated_traffic_flow_data.csv")

In [5]:
tf_df.head()

,dateObservedFrom,dateObservedTo,id,location,intensity,ocupancy
0,{'value': '2022-05-01T01:00:00+01:00'},{'value': '2022-05-01T01:10:00+01:00'},urn:ngsi-ld:TrafficFlowObserved:porto:traffic:...,"{'value': {'coordinates': [41.147012, -8.63282...",{'value': 90},{'value': 6}
1,{'value': '2022-05-01T01:00:00+01:00'},{'value': '2022-05-01T01:10:00+01:00'},urn:ngsi-ld:TrafficFlowObserved:porto:traffic:...,"{'value': {'coordinates': [41.147012, -8.63282...",{'value': 270},{'value': 14}
2,{'value': '2022-05-01T01:00:00+01:00'},{'value': '2022-05-01T01:10:00+01:00'},urn:ngsi-ld:TrafficFlowObserved:porto:traffic:...,"{'value': {'coordinates': [41.147012, -8.63282...",{'value': 6},{'value': 0}
3,{'value': '2022-05-01T01:10:00+01:00'},{'value': '2022-05-01T01:20:00+01:00'},urn:ngsi-ld:TrafficFlowObserved:porto:traffic:...,"{'value': {'coordinates': [41.147012, -8.63282...",{'value': 108},{'value': 7}
4,{'value': '2022-05-01T01:10:00+01:00'},{'value': '2022-05-01T01:20:00+01:00'},urn:ngsi-ld:TrafficFlowObserved:porto:traffic:...,"{'value': {'coordinates': [41.147012, -8.63282...",{'value': 306},{'value': 28}


In [35]:
# Convert date strings to datetime objects
tf_df['dateFrom'] = pd.to_datetime(tf_df['dateObservedFrom'].apply(lambda x: x.split("'")[3]))
tf_df['dateTo'] = pd.to_datetime(tf_df['dateObservedTo'].apply(lambda x: x.split("'")[3]))

# Remove timezone information
tf_df["dateTo"] = tf_df["dateTo"].apply(lambda x: x.tz_localize(None))
tf_df["dateFrom"] = tf_df["dateFrom"].apply(lambda x: x.tz_localize(None))

# Define your date range
start_date = pd.to_datetime('2022-07-01')
end_date = pd.to_datetime('2022-12-31')

# Filter the DataFrame
filtered_df = tf_df[(tf_df['dateFrom'] >= start_date) & (tf_df['dateTo'] <= end_date)]


/tmp/ipykernel_998226/3250462592.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  tf_df['dateFrom'] = pd.to_datetime(tf_df['dateObservedFrom'].apply(lambda x: x.split("'")[3]))
/tmp/ipykernel_998226/3250462592.py:3: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  tf_df['dateTo'] = pd.to_datetime(tf_df['dateObservedTo'].apply(lambda x: x.split("'")[3]))


In [45]:
filtered_df["processed_id"] = filtered_df["id"].apply(lambda x: x.split(":")[-1])

/tmp/ipykernel_998226/2744385253.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["processed_id"] = filtered_df["id"].apply(lambda x: x.split(":")[-1])


In [65]:
def get_latitude(row):
    return float(row.split(":")[-1].split(",")[0].split("[")[-1])

In [66]:
def get_longitude(row):
    return float(row.split(":")[-1].split(",")[-1].split("]")[0])

In [67]:
filtered_df["longitude"] = filtered_df["location"].apply(lambda x: get_longitude(x))
filtered_df["latitude"] = filtered_df["location"].apply(lambda x: get_latitude(x))

/tmp/ipykernel_998226/3900067009.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["longitude"] = filtered_df["location"].apply(lambda x: get_longitude(x))
/tmp/ipykernel_998226/3900067009.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["latitude"] = filtered_df["location"].apply(lambda x: get_latitude(x))


In [76]:
def get_values(row):
    return int(row.split(":")[-1].split("}")[0])

In [79]:
filtered_df["intensity"] = filtered_df["intensity"].apply(lambda x: get_values(x))
filtered_df["ocupancy"] = filtered_df["ocupancy"].apply(lambda x: get_values(x))

/tmp/ipykernel_998226/2314187766.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["intensity"] = filtered_df["intensity"].apply(lambda x: get_values(x))
/tmp/ipykernel_998226/2314187766.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["ocupancy"] = filtered_df["ocupancy"].apply(lambda x: get_values(x))


In [81]:
clean_df = filtered_df[["dateFrom", "dateTo",
                        "processed_id", "longitude",
                        "latitude", "intensity", "ocupancy"]]

In [84]:
clean_df.to_csv("../data/processed/traffic_flow_clean.csv", index=False)